In [2]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.tree import RandomForest, RandomForestModel

from pyspark.mllib.util import MLUtils

%pylab inline

import numpy as np
from time import time

%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [1]:
from pyspark import SparkContext

sc=SparkContext()

In [3]:
from numpy.random import rand
p=0.9
data=[]
for i in range(10):
    for j in range(10):
        if np.abs(i-4)<3 and np.abs(j-6)<3:
            y=2*(rand()<p)-1
        else:
            y=2*(rand()>p)-1
        print "%1.0f "%((1+y)/2),
        data.append(LabeledPoint(y,[i,j]))
    print

0  0  0  0  1  0  0  0  0  0 
0  0  0  0  0  0  0  0  0  1 
0  0  0  0  1  1  1  0  0  0 
0  1  0  1  1  1  1  1  0  0 
1  0  0  0  0  1  1  1  1  0 
0  1  1  0  1  0  1  0  1  0 
0  0  0  1  1  1  0  1  1  0 
1  0  0  0  0  0  0  0  0  0 
0  0  0  0  0  0  0  0  0  0 
0  0  0  0  0  0  0  0  0  0 


In [4]:
dataRDD=sc.parallelize(data)
dataRDD.getNumPartitions()

8

In [5]:
from DistributedBoosting import *

Populating the interactive namespace from numpy and matplotlib


In [6]:
dataRDD.first()

LabeledPoint(-1.0, [0.0,0.0])

In [7]:
Data=dataRDD

In [8]:
    T=Timer()
    T.stamp('Started')

    X=Data.first()
    feature_no=len(X.features)
    partition_no=Data.getNumPartitions()
    if partition_no != feature_no:
        Data=Data.repartition(feature_no).cache()

    # Split data into training and test
    (trainingData,testData)=Data.randomSplit([0.7,0.3])
    print 'Sizes: Data1=%d, trainingData=%d, testData=%d'%      (Data.count(),trainingData.cache().count(),testData.cache().count())
    T.stamp('Split into train and test')
    # Glom each partition into a local array
    G=trainingData.glom()
    GTest=testData.glom()  
    T.stamp('glom')

    # Add an index to each partition to identify it.
    def f(splitIndex, iterator): yield splitIndex,iterator.next()
    GI=G.mapPartitionsWithIndex(f)
    GTI=GTest.mapPartitionsWithIndex(f)
    T.stamp('add partition index')



Sizes: Data1=100, trainingData=63, testData=37


In [9]:
GI.getNumPartitions()

2

In [11]:
GI.first()[1][:5]

[LabeledPoint(-1.0, [0.0,0.0]),
 LabeledPoint(-1.0, [0.0,6.0]),
 LabeledPoint(-1.0, [0.0,7.0]),
 LabeledPoint(-1.0, [0.0,8.0]),
 LabeledPoint(-1.0, [0.0,9.0])]

In [12]:
Prepare_partition_data_structure(GI.first())

{'feature_values': array([[ 0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,  3.,
          4.,  4.,  4.,  4.,  4.,  4.,  5.,  5.,  5.,  5.,  5.,  5.,  6.,
          6.,  6.,  6.,  6.,  6.,  6.,  7.,  7.,  7.,  7.,  7.,  7.,  8.,
          8.,  8.,  8.,  8.,  8.,  8.,  9.,  9.]]),
 'index': 0,
 'labels': array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1.,
        -1., -1., -1.,  1.,  1., -1., -1., -1., -1.,  1.,  1., -1., -1.,
        -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,
        -1.,  1.,  1., -1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1.]),
 'weights': array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  

In [11]:
    # Prepare the data structure for each partition.
    GR=GI.map(Prepare_partition_data_structure)
    print 'number of elements in GR=', GR.cache().count()
    T.stamp('Prepare_partition_data_structure')



number of elements in GR=

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 11.0 failed 1 times, most recent failure: Lost task 1.0 in stage 11.0 (TID 18, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/yoavfreund/spark-latest/python/lib/pyspark.zip/pyspark/worker.py", line 98, in main
    command = pickleSer._read_with_length(infile)
  File "/Users/yoavfreund/spark-latest/python/lib/pyspark.zip/pyspark/serializers.py", line 164, in _read_with_length
    return self.loads(obj)
  File "/Users/yoavfreund/spark-latest/python/lib/pyspark.zip/pyspark/serializers.py", line 422, in loads
    return pickle.loads(obj)
  File "DistributedBoosting.py", line 9, in <module>
    get_ipython().magic(u'pylab inline')
NameError: name 'get_ipython' is not defined

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.sca

In [13]:
feature_no

2